In [1]:
import os

In [2]:
%pwd

'c:\\Users\\MAINGATE\\Desktop\\Maize_disease_classification\\end-to-end_mazie_disease_classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\MAINGATE\\Desktop\\Maize_disease_classification\\end-to-end_mazie_disease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir : Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):

        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config= self.config.prepare_callbacks

        model_ckpt_dir=os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])


        prepare_callback_config= PrepareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)

        )
        
        return prepare_callback_config

In [ ]:
import os

import tf_keras as tf
import time


[2025-05-04 06:31:14,284: WARNING: module_wrapper: From c:\Users\MAINGATE\.conda\envs\maize\lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [14]:
class PrepareCallback:
    def __init__(self, config:PrepareCallbacksConfig):
        self.config= config




    @property
    def _create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir=os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.callbacks.TensorBoard(log_dir=tb_running_log_dir)


    @property
    def _create_ckpt_callbacks(self):
        return tf.callbacks.ModelCheckpoint(
            filepath= str(self.config.checkpoint_model_filepath),
            save_best_only= True
        )


    def get_tb_ckpt_callbacks(self):
        return [ 
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [15]:
try:
    config= ConfigurationManager()
    prepare_callbacks_config=config.get_prepare_callback_config()
    prepare_callbacks= PrepareCallback(config=prepare_callbacks_config)
    callback_list= prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2025-05-04 06:46:05,292: INFO: common: yaml file: (path_to_yaml) loaded successfully]
[2025-05-04 06:46:05,417: INFO: common: yaml file: (path_to_yaml) loaded successfully]
[2025-05-04 06:46:05,444: INFO: common: created a directory at : artifacts]
[2025-05-04 06:46:05,495: INFO: common: created a directory at : artifacts\prepare_callbacks\tensorboard_log_dir]


TypeError: __init__() missing 1 required positional argument: 'filepath'